## Import libraries

In [1]:
import os
import json

from llama_index.core import (
    VectorStoreIndex,
    Settings,
    Document,
    StorageContext,
)
from llama_index.vector_stores.chroma import ChromaVectorStore
from langchain_huggingface import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index.llms.ollama import Ollama

from llama_index.core.chat_engine.types import ChatMode
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core import ChatPromptTemplate

from prompts import SYSTEM_PROMPT, USER_PROMPT

## LLM + Embeddings

In [31]:
llm = Ollama(
    model='llama3.1:8b',
    )

lc_embed_model = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-small"
)
embed_model = LangchainEmbedding(lc_embed_model)

Settings.llm = llm
Settings.embed_model = embed_model

In [36]:
import os
from dotenv import load_dotenv
from llama_index.llms.openai import OpenAI

load_dotenv()
# Set the following API Keys in the Python environment. Will be used later.
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

Settings.llm = OpenAI(temperature=0, model="gpt-4o")


## Vector store

In [3]:
import chromadb

chroma_client = chromadb.PersistentClient(path="../chromadb")
chroma_collection = chroma_client.get_collection("unit1_db")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=embed_model,
)

# Query Pipeline

## Tạo QA Template

In [13]:
chat_text_qa_msgs = [
    ("system", SYSTEM_PROMPT),
    ("user", USER_PROMPT),
]
text_qa_template = ChatPromptTemplate.from_messages(chat_text_qa_msgs)

## Khởi tạo Chat Engine

In [28]:
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.schema import NodeWithScore, QueryBundle
from llama_index.core.vector_stores.types import VectorStoreQuery
from typing import List

class CustomRetriever(BaseRetriever):
    def __init__(self, vector_index, embed_model, top_k=4):
        self.vector_index = vector_index
        self.embed_model = embed_model
        self.top_k = top_k
        
    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        # Tùy chỉnh query text trước khi embedding
        modified_query = f"query: {query_bundle.query_str}"
        
        # Tạo embedding cho query
        query_embedding = self.embed_model.get_text_embedding(modified_query)
        
        # Tạo vector store query
        vector_store_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self.top_k
        )
        
        # Thực hiện tìm kiếm vector
        query_result = self.vector_index.vector_store.query(
            query=vector_store_query
        )
        
        # Chuyển đổi kết quả thành List[NodeWithScore]
        nodes_with_scores = []
        for node, score in zip(query_result.nodes, query_result.similarities or []):
            nodes_with_scores.append(NodeWithScore(node=node, score=score))
            
        return nodes_with_scores
    
from llama_index.core.chat_engine import ContextChatEngine
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core.chat_engine.types import AgentChatResponse
from llama_index.core.tools import ToolOutput

class CustomChatEngine(ContextChatEngine):
    """Custom chat engine that extends ContextChatEngine"""
    def __init__(
        self,
        retriever,
        llm,
        memory,
        text_qa_template,
        **kwargs
    ):
        # Tạo prefix_messages từ system prompt trong template
        system_prompt = text_qa_template.message_templates[0].blocks[0].text
        prefix_messages = [
            ChatMessage(content=system_prompt, role=llm.metadata.system_role)
        ]
        
        super().__init__(
            retriever=retriever,
            llm=llm,
            memory=memory,
            prefix_messages=prefix_messages,
            **kwargs
        )
        self.text_qa_template = text_qa_template

    def _is_answer(self, message: str) -> bool:
        """Kiểm tra xem message có phải là câu trả lời không"""
        # Lấy lịch sử chat
        chat_history = self._memory.get()
        if not chat_history:
            return False
            
        # Lấy tin nhắn cuối cùng của AI
        last_ai_message = None
        for msg in reversed(chat_history):
            if msg.role == MessageRole.ASSISTANT:
                last_ai_message = msg.content
                break
                
        if not last_ai_message:
            return False
            
        # Tạo prompt để phân tích
        analysis_prompt = f"""Analyze if the following user message is an answer to the previous question/exercise:
            Previous AI message: {last_ai_message}
            User message: {message}

            Return only "true" if it's an answer, or "false" if it's a new question/request.
        """

        # Phân tích với LLM
        response = self._llm.complete(analysis_prompt)
        print(response)
        return "false" not in str(response).strip().lower()

    def chat(self, message: str) -> AgentChatResponse:
        # Kiểm tra xem message có phải là câu trả lời không
        is_answer = self._is_answer(message)
        print(is_answer)
        
        # Chỉ truy xuất nodes nếu không phải là câu trả lời
        if not is_answer:
            retrieved_nodes = self._get_nodes(message)
        else:
            retrieved_nodes = []
        
        # Tạo context từ retrieved documents
        context_str = "\n\n".join([node.text for node in retrieved_nodes])
        
        # Tạo prompt với template
        prompt = self.text_qa_template.format(
            context_str=context_str,
            chat_history=self._memory.get(),
            question=message
        )
        
        # Gọi LLM để generate response
        response = self._llm.complete(prompt)
        
        # Cập nhật memory
        user_message = ChatMessage(role=MessageRole.USER, content=message)
        ai_message = ChatMessage(role=MessageRole.ASSISTANT, content=str(response))
        self._memory.put(user_message)
        self._memory.put(ai_message)
        
        # Trả về AgentChatResponse
        return AgentChatResponse(
            response=str(response),
            sources=[
                ToolOutput(
                    tool_name="retriever",
                    content=str(retrieved_nodes),
                    raw_input={"message": message},
                    raw_output=retrieved_nodes,
                )
            ],
            source_nodes=retrieved_nodes,
        )

In [37]:
memory = ChatMemoryBuffer.from_defaults(token_limit=2048)

# Khởi tạo custom retriever
custom_retriever = CustomRetriever(
    vector_index=index,
    embed_model=embed_model,
    top_k=8
)

# Khởi tạo custom chat engine
chat_engine = CustomChatEngine(
    retriever=custom_retriever,
    llm=llm,
    memory=memory,
    text_qa_template=text_qa_template
)

## Hỏi đáp

In [72]:
# Test the RAG pipeline
def ask_question(question):
    response = chat_engine.chat(question)
    print("Q:", question)
    print("\nA:", response)
    print(f"\nTotal Retrieved Nodes: {len(response.source_nodes)}")
    print("\nRetrieved Nodes:")
    
    for i, node in enumerate(response.source_nodes, 1):
        print(f"\nNode #{i}")
        print(f"Score: {node.score:.4f}")
        print(f"ID: {node.metadata.get('id')}")
        print(f"Unit: {node.metadata.get('unit')}")
        print(f"Section: {node.metadata.get('section')}")
        print(f"Type: {node.metadata.get('type', 'N/A')}")
        print(f"Content Preview: {node.text[:200]}...")
        print("-" * 80)

In [ ]:
# questions = [
#     "Hãy cho tôi 10 câu hỏi từ nội dung đoạn hội thoại trong phần Getting Started Unit 1 để tôi ôn tập",
# ]

# for q in questions:
#     ask_question(q)
#     print("\n" + "-"*80 + "\n")


In [35]:
def interactive_chat():
    print("Bắt đầu chat với AI Assistant (gõ 'quit' hoặc 'exit' để thoát)")
    print("-" * 50)
    
    # Nhận input từ người dùng
    user_input = "Em muốn cô tạo ra 10 câu điền từ để ôn tập về từ vựng trong Unit 1"

    # Xử lý câu trả lời
    response = chat_engine.chat(user_input)
    
    # In kết quả
    print("\nGiáo viên:", response.response)
    
    user_input = "1. A"
    
    # Xử lý câu trả lời
    response = chat_engine.chat(user_input)
    
    # In kết quả
    print("\nGiáo viên:", response.response)

# Sử dụng hàm
interactive_chat()

Bắt đầu chat với AI Assistant (gõ 'quit' hoặc 'exit' để thoát)
--------------------------------------------------
False

The user message is asking for the creation of 10 fill-in-the-blank exercises for vocabulary practice in Unit 1, which is unrelated to the previous multiple-choice question. It does not attempt to answer the original question.
False

Giáo viên: Xin chào các em! Hôm nay, chúng ta sẽ tập trung vào từ vựng trong phần "Life stories we admire". Hãy đọc kỹ và chọn đáp án đúng!

**Câu hỏi 1:** A _________ descriptions of things that have happened.

A) accounts
B) death
C) devoting
D) youth

Hãy chọn đáp án đúng!
True
True

Giáo viên: Tôi xin chào mừng các em học sinh lớp 12! Hôm nay, chúng ta sẽ tập trung vào việc cải thiện kỹ năng từ vựng bằng cách giải quyết các bài tập liên quan.

Bài tập đầu tiên:

Multiple Choice Question:

What is the meaning of the word "skeptical" in the following sentence?

"The tourists were skeptical about the local legend."

A) tin tưởng
B) nghi

In [38]:
def interactive_chat():
    print("Bắt đầu chat với AI Assistant (gõ 'quit' hoặc 'exit' để thoát)")
    print("-" * 50)
    
    # Nhận input từ người dùng
    user_input = "Em muốn cô tạo ra 10 câu điền từ để ôn tập về từ vựng trong Unit 1"

    # Xử lý câu trả lời
    response = chat_engine.chat(user_input)
    
    # In kết quả
    print("\nGiáo viên:", response.response)
    
    user_input = "1. A"
    
    # Xử lý câu trả lời
    response = chat_engine.chat(user_input)
    
    # In kết quả
    print("\nGiáo viên:", response.response)

# Sử dụng hàm
interactive_chat()

Bắt đầu chat với AI Assistant (gõ 'quit' hoặc 'exit' để thoát)
--------------------------------------------------
False

Giáo viên: Let's begin our vocabulary exercise based on the passage.

**Câu hỏi 1:** Complete the sentence:
Dang Thuy Tram wrote about/operated on injured soldiers during the war.

A) She gave medical treatment to them.
B) She was a doctor who operated on many patients.
C) She kept their stories in her diary.
D) She helped those soldiers by writing about them.

Hãy chọn đáp án đúng!
False

The user is answering the exercise with option A), but the format and phrasing of their response ("1. A") suggests they are unsure about how to submit their answer, rather than providing additional context or asking a new question.
False

Giáo viên: Được rồi! Let's start with a multiple-choice question.

**Question 1:** What does "accounts" mean in the context of life stories?

A) Mô tả về một người
B) Những câu chuyện về cuộc sống
C) Những bài viết về các sự kiện quan trọng
D) Các